In [13]:
### all together ########################
import stanza
import spacy
import glob
import pandas as pd

import stanza.pipeline
stanza.download('en')
nlp_stanza = stanza.Pipeline('en', processors='tokenize,ner') 
nlp_spacy = spacy.load('en_core_web_sm')

path = './Biographies/*'
files = sum([glob.glob(dir+'/*.txt') for dir in glob.glob(path)],[]) # a commenter pour tout faire
files = files[:3]

2024-06-12 12:05:00 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-12 12:05:00 INFO: Downloading default packages for language: en (English) ...
2024-06-12 12:05:02 INFO: File exists: /home/marina/stanza_resources/en/default.zip
2024-06-12 12:05:08 INFO: Finished downloading models and saved to /home/marina/stanza_resources
2024-06-12 12:05:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-12 12:05:08 INFO: Downloaded file to /home/marina/stanza_resources/resources.json
2024-06-12 12:05:08 WARNING: Language en package default expects mwt, which has been added
2024-06-12 12:05:09 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner

In [21]:
def extract_ents_stanza(file):
    with open(file, 'r') as f:
        content = f.read()
        doc = nlp_stanza(content)
        ents = [(ent.text, ent.type) for ent in doc.ents]
    return ents

def extract_ents_spacy(file):
    with open(file, 'r') as f:
        content = f.read()
        doc = nlp_spacy(content)
        ents = [(ent.text, ent.label_) for ent in doc.ents]
    return ents

def store_ents(files):
    data = []
    for file in files:
        file_name = file.split('/')[-1]
        data.append({'file_name': file_name, 
                     'ents_stanza': extract_ents_stanza(file), 
                     'ents_spacy': extract_ents_spacy(file)})
    return data

data = store_ents(files)
df = pd.DataFrame(data)
df.head()

,file_name,ents_stanza,ents_spacy
0,AquileoJ.Echeverría_Journalists.txt,"[(Aquileo J. Echeverría, PERSON), (May 22, 186...","[(Aquileo J. Echeverría, PERSON), (May 22, 186..."
1,ThomasHenryShadwellClerke_Journalists.txt,"[(Thomas Henry Shadwell Clerke, PERSON), (KH, ...","[(Thomas Henry Shadwell Clerke, PERSON), (KH, ..."
2,Journalist_Journalists.txt,"[(Matthew C. Nisbet, PERSON), (Walter Lippmann...","[(Matthew C. Nisbet, PERSON), (Walter Lippmann..."


In [41]:
stanza_categories =  []
for ents in df['ents_stanza']:
    for ent in ents:
        label = ent[1]
        stanza_categories.append(label)
stanza_categories = list(set(stanza_categories))
print(stanza_categories)
stanza_ctgs = list(set([ent[1] for ent in ents for ents in df['ents_stanza']]))
print(stanza_ctgs)
print(len(stanza_categories))
print(len(stanza_ctgs))
print(stanza_categories-stanza_ctgs)

['LANGUAGE', 'PERSON', 'ORDINAL', 'PERCENT', 'FAC', 'DATE', 'EVENT', 'LOC', 'WORK_OF_ART', 'GPE', 'NORP', 'ORG', 'CARDINAL']
['PERSON', 'ORDINAL', 'PERCENT', 'FAC', 'DATE', 'EVENT', 'WORK_OF_ART', 'GPE', 'NORP', 'ORG', 'CARDINAL']
13
11


TypeError: unsupported operand type(s) for -: 'list' and 'list'